#### Parametri da modificare:

In [1]:
# Specificare la cartella da analizzare con i programmi sas
cartella_da_esplorare = r"C:\Miguel\progetti\05_Modifica Testi\01_data\TEST\PROVA ELIMINA"
estensione_file = ".sas"

# Specificare il percorso del file Excel di output
excel_file_path = r"C:\Miguel\progetti\05_Modifica Testi\02_output\ReportPrincipale.xlsx"


# Funzioni per estrarre testo tra due keywords

***INPUT***

cartella_da_esplorare: cartella principale da analizzare

estensione: estensione del file da elaborare

keyword: parola chiave da cui iniziare ad estrarre il testo

stop_keyword: parola chiave in cui finisce l'estrazione

***FASI***
1) La funzione **crea_dataframe_presenza_parole** itera attraverso le cartelle, incluse le sottocartelle, alla ricerca di programmi con estensione .sas.
2) Per ciascun file .sas individuato, la funzione esamina tutte le righe del programma.
3) La funzione **elimina_parole_tra_commenti** elimina il testo compreso tra i commenti.
4) La funzione **estrai_blocchi_tra_keywords** estrae tutto il testo compresso tra le due keywords.
5) Infine la funzione **processa_cartella** restituisce un dataframe contenente il nome del programma, il percorso e una lista con tutti i blocchi di testo individuati

Successivamente l'output della funzione processa_cartella viene ulteriormente elaborata in modo da estrarre le parti di interesse. 

In [2]:
import os
import pandas as pd
import re

def elimina_parole_tra_commenti(line, in_comment_block):
    if in_comment_block:
        end_comment = line.find('*/')
        if end_comment != -1:
            line = line[end_comment + 2:]
            in_comment_block = False
        else:
            return '', True  # continua a leggere la prossima riga

    start_comment = line.find('/*')
    while start_comment != -1:
        end_comment = line.find('*/', start_comment + 2)
        if end_comment != -1:
            line = line[:start_comment] + line[end_comment + 2:]
            start_comment = line.find('/*')
        else:
            in_comment_block = True
            line = line[:start_comment]
            break

    return line, in_comment_block
    
def estrai_blocchi_tra_keywords(percorso_file, keyword, stop_keyword):
    righe_estratte = []
    blocco_corrente = ""
    in_comment_block = False
    with open(percorso_file, 'r') as file:
        for riga in file:
            line_originale = riga.strip()
            riga, in_comment_block = elimina_parole_tra_commenti(line_originale, in_comment_block)

            
            if keyword.lower() in riga.lower() and stop_keyword.lower() in riga.lower():
                righe_estratte.append(riga.strip())
            # Controlla se la keyword è presente nella riga (non case-sensitive)
            elif keyword.lower() in riga.lower():
                blocco_corrente += riga.strip() + ' '

                # Aggiungi la riga corrente se la stop_keyword è presente nella stessa riga
                if stop_keyword.lower() in riga.lower():
                    righe_estratte.append(blocco_corrente.strip())
                    blocco_corrente = ""
            # Aggiungi la riga corrente al blocco se stai già estraendo
            elif blocco_corrente:
                blocco_corrente += riga.strip() + ' '
                # Aggiungi il blocco alla lista se trovi la stop_keyword
                if stop_keyword.lower() in riga.lower():
                    righe_estratte.append(blocco_corrente.strip())
                    blocco_corrente = ""

    return righe_estratte

def processa_cartella(cartella_da_esplorare, estensione, keyword, stop_keyword):
    righe_totali = []
    dati_temporanei = {'Nome File': [], 'Percorso File' : [], keyword: []}
    
    for root, dirs, files in os.walk(cartella_da_esplorare):
        for file_name in files:
            if file_name.endswith(estensione):
                file_path = os.path.join(root, file_name)
                
                righe_blocchi_import = estrai_blocchi_tra_keywords(file_path, keyword, stop_keyword)
                # righe_totali.extend(righe_blocchi_import)

                dati_temporanei['Nome File'].append(file_name)
                dati_temporanei['Percorso File'].append(root)
                dati_temporanei[keyword].append(righe_blocchi_import)

    dataframe = pd.DataFrame(dati_temporanei)
    dataframe.set_index('Nome File', inplace=True)
    return dataframe

#### PROC EXPORT

In [3]:
keyword = 'proc export'
stop_keyword = 'run;'

righe_totali = processa_cartella(cartella_da_esplorare, estensione_file, keyword, stop_keyword)

In [4]:
df_exploded = righe_totali.explode("proc export")

# Elimina le righe con valori mancanti nella colonna "proc export"
df_exploded = df_exploded.dropna(subset=["proc export"])

# df_exploded = df_exploded[~df_exploded["proc export"].str.lower().str.startswith("/*proc")]
df_export = df_exploded[~(df_exploded["proc export"].str.lower().str.startswith("/*proc") | df_exploded["proc export"].str.lower().str.endswith("run;*/"))]

In [5]:
df_export['Statement DATA='] = df_export["proc export"].apply(lambda x: re.search(r'data\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'DATA\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_export['Statement OUTFILE='] = df_export["proc export"].apply(lambda x: re.search(r'(OUTFILE|FILE)\s*=\s*(\S+)', x, flags=re.IGNORECASE).group(2) if re.search(r'(OUTFILE|FILE)\s*=\s*(\S+)', x, flags=re.IGNORECASE) else None)
df_export['Statement SHEET='] = df_export["proc export"].apply(lambda x: re.search(r'SHEET\s*=\s*(\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'SHEET\s*=\s*(\S+)', x, flags=re.IGNORECASE) else "")
df_export['Statement DBMS='] = df_export["proc export"].apply(lambda x: re.search(r'DBMS\s*=\s*(\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'DBMS\s*=\s*(\S+)', x, flags=re.IGNORECASE) else None)

#### PROC IMPORT

In [6]:
keyword = 'proc import'
stop_keyword = 'run;'

righe_totali = processa_cartella(cartella_da_esplorare, estensione_file, keyword, stop_keyword)

In [7]:
df_exploded = righe_totali.explode("proc import")

df_exploded = df_exploded.dropna(subset=["proc import"])

df_import = df_exploded[~(df_exploded["proc import"].str.lower().str.startswith("/*proc") | df_exploded["proc import"].str.lower().str.endswith("run;*/"))]

In [8]:
df_import['Statement OUT='] = df_import["proc import"].apply(lambda x: re.search(r'OUT\s*=\s*(\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'OUT\s*=\s*(\S+)', x, flags=re.IGNORECASE) else None)
df_import['Statement DATAFILE='] = df_import["proc import"].apply(lambda x: re.search(r'(DATAFILE|FILE)\s*=\s*(\S+)', x, flags=re.IGNORECASE).group(2) if re.search(r'(DATAFILE|FILE)\s*=\s*(\S+)', x, flags=re.IGNORECASE) else None)
df_import['Statement DBMS='] = df_import["proc import"].apply(lambda x: re.search(r'DBMS\s*=\s*(\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'DBMS\s*=\s*(\S+)', x, flags=re.IGNORECASE) else None)

#### %EMAIL

In [9]:
keyword = '%email'
stop_keyword = ');'

righe_totali = processa_cartella(cartella_da_esplorare, estensione_file, keyword, stop_keyword)

In [10]:
df_exploded = righe_totali.explode("%email")

df_exploded = df_exploded.dropna(subset=["%email"])
df_exploded["%email"] = df_exploded["%email"].str.lower()
df_email = df_exploded[~(df_exploded["%email"].str.startswith("/*%email") | df_exploded["%email"].str.endswith(");*/"))]

In [11]:
df_email['subject='] = df_email["%email"].apply(lambda x: re.search(r'subject\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'subject\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)

df_email['to_addr1='] = df_email["%email"].apply(lambda x: re.search(r'to_addr1\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr1\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr2='] = df_email["%email"].apply(lambda x: re.search(r'to_addr2\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr2\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr3='] = df_email["%email"].apply(lambda x: re.search(r'to_addr3\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr3\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr4='] = df_email["%email"].apply(lambda x: re.search(r'to_addr4\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr4\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr5='] = df_email["%email"].apply(lambda x: re.search(r'to_addr5\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr5\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr6='] = df_email["%email"].apply(lambda x: re.search(r'to_addr6\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr6\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr7='] = df_email["%email"].apply(lambda x: re.search(r'to_addr7\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr7\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr8='] = df_email["%email"].apply(lambda x: re.search(r'to_addr8\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr8\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr9='] = df_email["%email"].apply(lambda x: re.search(r'to_addr9\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr9\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['to_addr10='] = df_email["%email"].apply(lambda x: re.search(r'to_addr10\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'to_addr10\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)

df_email['c_addr1='] = df_email["%email"].apply(lambda x: re.search(r'c_addr1\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr1\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr2='] = df_email["%email"].apply(lambda x: re.search(r'c_addr2\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr2\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr3='] = df_email["%email"].apply(lambda x: re.search(r'c_addr3\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr3\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr4='] = df_email["%email"].apply(lambda x: re.search(r'c_addr4\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr4\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr5='] = df_email["%email"].apply(lambda x: re.search(r'c_addr5\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr5\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr6='] = df_email["%email"].apply(lambda x: re.search(r'c_addr6\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr6\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr7='] = df_email["%email"].apply(lambda x: re.search(r'c_addr7\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr7\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr8='] = df_email["%email"].apply(lambda x: re.search(r'c_addr8\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr8\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr9='] = df_email["%email"].apply(lambda x: re.search(r'c_addr9\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr9\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)
df_email['c_addr10='] = df_email["%email"].apply(lambda x: re.search(r'c_addr10\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'c_addr10\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)

df_email['attach_file1='] = df_email["%email"].apply(lambda x: re.search(r'attach_file1\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE).group(1) if re.search(r'attach_file1\s*=\s*(\S+|\S+\s+\S+)', x, flags=re.IGNORECASE) else None)

df_email['message1='] = df_email["%email"].apply(lambda x: re.search(r'message1\s*=\s*((?:(?!message2).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message1\s*=\s*((?:(?!message2).)*)', x, flags=re.IGNORECASE) else None)
df_email['message2='] = df_email["%email"].apply(lambda x: re.search(r'message2\s*=\s*((?:(?!message3).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message2\s*=\s*((?:(?!message3).)*)', x, flags=re.IGNORECASE) else None)
df_email['message3='] = df_email["%email"].apply(lambda x: re.search(r'message3\s*=\s*((?:(?!message4).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message3\s*=\s*((?:(?!message4).)*)', x, flags=re.IGNORECASE) else None)
df_email['message4='] = df_email["%email"].apply(lambda x: re.search(r'message4\s*=\s*((?:(?!message5).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message4\s*=\s*((?:(?!message5).)*)', x, flags=re.IGNORECASE) else None)
df_email['message5='] = df_email["%email"].apply(lambda x: re.search(r'message5\s*=\s*((?:(?!message6).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message5\s*=\s*((?:(?!message6).)*)', x, flags=re.IGNORECASE) else None)
df_email['message6='] = df_email["%email"].apply(lambda x: re.search(r'message6\s*=\s*((?:(?!message7).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message6\s*=\s*((?:(?!message7).)*)', x, flags=re.IGNORECASE) else None)
df_email['message7='] = df_email["%email"].apply(lambda x: re.search(r'message7\s*=\s*((?:(?!message8).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message7\s*=\s*((?:(?!message8).)*)', x, flags=re.IGNORECASE) else None)
df_email['message8='] = df_email["%email"].apply(lambda x: re.search(r'message8\s*=\s*((?:(?!message9).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message8\s*=\s*((?:(?!message9).)*)', x, flags=re.IGNORECASE) else None)
df_email['message9='] = df_email["%email"].apply(lambda x: re.search(r'message9\s*=\s*((?:(?!message10).)*)', x, flags=re.IGNORECASE).group(1) if re.search(r'message9\s*=\s*((?:(?!message10).)*)', x, flags=re.IGNORECASE) else None)

#Elimino tutte le colonne con solo missing
df_email = df_email.dropna(axis=1, how='all')
columns_address = [col for col in df_email.columns if 'to_addr' in col or 'c_addr' in col]

df_email[columns_address] = df_email[columns_address].apply(lambda x: x.str.replace(',', ''))

In [12]:
columns_message = [col for col in df_email.columns if 'message' in col]

df_email['Testo mail'] = df_email[columns_message].apply(lambda x: ''.join(x.dropna().astype(str)), axis=1)
df_email.drop(columns=columns_message, inplace=True)

### Export Output Finale

In [13]:
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a') as writer:
    # Aggiungere un altro DataFrame a un file esistente
    df_export.reset_index().to_excel(writer, sheet_name='PROC EXPORT', index=False)
    df_import.reset_index().to_excel(writer, sheet_name='PROC IMPORT', index=False)
    df_email.reset_index().to_excel(writer, sheet_name='%EMAIL', index=False)

# ---------------------------------------------------------

##### Estrarre mail univoche

In [14]:
df_mails = pd.melt(df_email, value_vars=columns_address, value_name='RECEIVER_MAIL')

In [15]:
df_mails = df_mails[['RECEIVER_MAIL']].dropna()

df_mails = df_mails.drop_duplicates(subset=['RECEIVER_MAIL'])

In [16]:
# Creare una nuova colonna 'nuova_variabile'
df_mails['ID_RECEIVER'] = df_mails['RECEIVER_MAIL'].apply(lambda x: x.split('@')[0].replace('.', '_') if '.' in x.split('@')[0] else x.split('@')[0])
df_mails['ID_RECEIVER'] = df_mails['ID_RECEIVER'].str.slice(0, 32)
df_mails['Tipo_mail'] = df_mails['RECEIVER_MAIL'].apply(lambda x: 'arval_persona' if '@arval' in x and '.' in x.split('@')[0] else ('arval_gruppo' if '@arval' in x else 'altro'))

In [17]:
# df_mails.to_csv(r"C:\Miguel\progetti\05_Modifica Testi\02_output\mails.csv", sep=";", index=False)

In [18]:
df_mails

,RECEIVER_MAIL,ID_RECEIVER,Tipo_mail
0,arviamministrazionebonifici@arval.it,arviamministrazionebonifici,arval_gruppo
2,daniele.maramai@arval.it,daniele_maramai,arval_persona
4,micheledavide.cipullo@arval.it,micheledavide_cipullo,arval_persona
5,arvi_team_network_support@arval.it,arvi_team_network_support,arval_gruppo
6,arvi_sales_efficiency@arval.it,arvi_sales_efficiency,arval_gruppo
7,giovanni.sannino@arval.it,giovanni_sannino,arval_persona
8,catia.ciccolini@arval.it,catia_ciccolini,arval_persona
9,carlo.marchetti@arval.it,carlo_marchetti,arval_persona
10,aosnational@arval.it,aosnational,arval_gruppo
11,arvi_cdi_b2c@arval.it,arvi_cdi_b2c,arval_gruppo


# ------------------------------------------------------------------------

##### Creazione struttura_tabelle_eml_sch automatico

##### ID_RECEIVER_TO

In [19]:
df_to_addr = df_email.loc[:,df_email.columns.str.contains('to_addr')].reset_index()
df_to_addrT = pd.melt(df_to_addr, id_vars="Nome File", var_name='Email', value_name='RECEIVER_MAIL')
df_to_addrT = df_to_addrT.dropna(subset=['RECEIVER_MAIL'])
df_to_addrT = df_to_addrT[df_to_addrT['RECEIVER_MAIL']!=""] # capire se è giusto!!!

df_to_addrT2 = pd.merge(df_to_addrT, df_mails, on='RECEIVER_MAIL', how='inner')
df_to_addrT2['Numero'] = df_to_addrT2['Email'].str.extract(r'(\d+)') + "A"
df_to_addrT2 = df_to_addrT2.rename(columns={"ID_RECEIVER":"ID_RECEIVER_TO"})
df_to_addrT2 = df_to_addrT2.sort_values(["Nome File","Numero"])

##### ID_RECEIVER_CC

In [20]:
df_c_addr = df_email.loc[:,df_email.columns.str.contains('c_addr')].reset_index()
df_c_addrT = pd.melt(df_c_addr, id_vars="Nome File", var_name='Email', value_name='RECEIVER_MAIL')
df_c_addrT = df_c_addrT.dropna(subset=['RECEIVER_MAIL'])
df_c_addrT = df_c_addrT[df_c_addrT['RECEIVER_MAIL']!=""] # capire se è giusto!!!

df_c_addrT2 = pd.merge(df_c_addrT, df_mails, on='RECEIVER_MAIL', how='inner')
df_c_addrT2['Numero'] = df_c_addrT2['Email'].str.extract(r'(\d+)') + "B"
df_c_addrT2 = df_c_addrT2.rename(columns={"ID_RECEIVER":"ID_RECEIVER_CC"})
df_c_addrT2 = df_c_addrT2.sort_values(["Nome File","Numero"])

In [21]:
df_mail_param = pd.concat([df_to_addrT2, df_c_addrT2], ignore_index=True)
# df_mail_param.drop(columns=["Email","RECEIVER_MAIL","Tipo_mail"])

In [22]:
df_mail_param = df_mail_param.sort_values(["Nome File","Numero"])
df_mail_param = df_mail_param.drop(columns=["Email","RECEIVER_MAIL","Tipo_mail", "Numero"])

In [23]:
# df_mail_param.to_csv("mail_parametrizzato.csv", index=False)

In [24]:
df_mail_param

,Nome File,ID_RECEIVER_TO,ID_RECEIVER_CC
0,0038_MAN_Ins_ChkReco3_v2.0.sas,daniele_maramai,NaN
54,0038_MAN_Ins_ChkReco3_v2.0.sas,NaN,arvidataofficedesk
1,0379_DRI_Lst_MidTerm_Tracciati_MonOffre_v2.0.sas,micheledavide_cipullo,NaN
55,0379_DRI_Lst_MidTerm_Tracciati_MonOffre_v2.0.sas,NaN,arvireportingrichieste
2,0399_DRI_Lst_DbDashboardQuattoruote_v11.0.sas,arvi_team_network_support,NaN
...,...,...,...
52,altra ccooppia meglio se eliminata.sas,arviamministrazionebonifici,NaN
53,altra ccooppia meglio se eliminata.sas,arviamministrazionebonifici,NaN
96,altra ccooppia meglio se eliminata.sas,NaN,arvidataofficedesk
97,altra ccooppia meglio se eliminata.sas,NaN,arvidataofficedesk


Export struttura_tabelle_eml_sch automatico

In [25]:
# excel_file_path = r"C:\Miguel\progetti\05_Modifica Testi\02_output\struttura_tabelle_eml_sch_AUTOMATICO.xlsx"
# with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
#     # Aggiungere un altro DataFrame a un file esistente
#     df_mails.to_excel(writer, sheet_name='MAIL_RECEIVER', index=False)
#     df_mail_param.to_excel(writer, sheet_name='MAIL_ENTITY', index=False)